In [1]:
import numpy as np
import pandas as pd

In [2]:
bill_summaries = pd.read_csv('../data/bill_summaries.csv')
bioinfo = pd.read_csv('../data/bioinfo.csv')
fec_ids = pd.read_csv('../data/fec_ids.csv')
ideology = pd.read_csv('../data/ideology.csv')
sponsored_legislation = pd.read_csv('../data/sponsored_legislation.csv')
terms = pd.read_csv('../data/terms.csv')
vote_compare = pd.read_csv('../data/vote_compare.csv')

1. Do each of the CSVs have primary keys? If so, what?

In [3]:
bill_summaries.columns

Index(['actionDate', 'actionDesc', 'currentChamber', 'currentChamberCode',
       'lastSummaryUpdateDate', 'text', 'updateDate', 'versionCode',
       'bill.congress', 'bill.number', 'bill.originChamber',
       'bill.originChamberCode', 'bill.title', 'bill.type',
       'bill.updateDateIncludingText', 'bill.url'],
      dtype='object')

In [13]:
bill_summaries[['bill.type', 'bill.number', 'versionCode']].duplicated().value_counts()

False    2751
Name: count, dtype: int64

In [4]:
bioinfo.columns

Index(['bioguide_id', 'Full name', 'Chamber', 'State', 'Party', 'District',
       'birthYear', 'image', 'Office address', 'Phone', 'Website'],
      dtype='object')

In [14]:
bioinfo[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [5]:
fec_ids.columns

Index(['bioguide_id', 'fec_id'], dtype='object')

In [15]:
fec_ids[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [6]:
ideology.columns

Index(['bioname', 'chamber', 'left_right_ideology', 'state_abbrev',
       'district_code', 'icpsr', 'bioguide_id', 'party'],
      dtype='object')

In [16]:
ideology[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [7]:
sponsored_legislation.columns

Index(['introducedDate', 'type', 'url', 'number', 'title', 'bioguide_id'], dtype='object')

In [25]:
sponsored_legislation[['url']].duplicated().value_counts()

False    14379
Name: count, dtype: int64

In [8]:
terms.columns

Index(['bioguide_id', 'chamber', 'congress', 'stateCode', 'startYear',
       'endYear', 'district'],
      dtype='object')

In [30]:
terms[['bioguide_id', 'chamber', 'congress']].duplicated().value_counts()

False    3257
Name: count, dtype: int64

In [9]:
vote_compare.columns

Index(['bioname', 'comparison_member', 'agree'], dtype='object')

In [31]:
vote_compare[['bioname', 'comparison_member']].duplicated().value_counts()

False    206040
Name: count, dtype: int64